## Captum

In [18]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from captum.attr import (
    GradientShap,
    DeepLift,
    DeepLiftShap,
    IntegratedGradients,
    LayerConductance,
    NeuronConductance,
    NoiseTunnel,
)

In [19]:
class ToyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(3, 3)
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(3, 2)

        # initialize weights and biases
        self.lin1.weight = nn.Parameter(torch.arange(-4.0, 5.0).view(3, 3))
        self.lin1.bias = nn.Parameter(torch.zeros(1,3))
        self.lin2.weight = nn.Parameter(torch.arange(-3.0, 3.0).view(2, 3))
        self.lin2.bias = nn.Parameter(torch.ones(1,2))

    def forward(self, input):
        print(input.shape)
        return self.lin2(self.relu(self.lin1(input)))

In [20]:
model = ToyModel()
model.eval()

ToyModel(
  (lin1): Linear(in_features=3, out_features=3, bias=True)
  (relu): ReLU()
  (lin2): Linear(in_features=3, out_features=2, bias=True)
)

In [21]:
toy_input = torch.rand(2, 3)
baseline = torch.zeros((2,3))

In [22]:
ig = IntegratedGradients(model)
attributions, delta = ig.attribute(toy_input, baseline, target=0, return_convergence_delta=True)
print('IG Attributions:', attributions)
print('Convergence Delta:', delta)

torch.Size([100, 3])
torch.Size([2, 3])
torch.Size([2, 3])
IG Attributions: tensor([[-1.8479, -0.3526, -1.5562],
        [ 0.0000, -2.2000, -4.2150]], dtype=torch.float64)
Convergence Delta: tensor([5.9605e-08, 1.1921e-07], dtype=torch.float64)


In [ ]:
gs = GradientShap(model)

# We define a distribution of baselines and draw `n_samples` from that
# distribution in order to estimate the expectations of gradients across all baselines
baseline_dist = torch.randn(5, 3, 480, 640) * 0.001
attributions, delta = gs.attribute(input_, stdevs=0.09, n_samples=4, baselines=baseline_dist,
                                   target=0, return_convergence_delta=True)
print('GradientShap Attributions:', attributions)
print('Convergence Delta:', delta)
print('Average delta per example:', torch.mean(delta.reshape(input.shape[0], -1), dim=1))

In [ ]:
dl = DeepLift(model)
attributions, delta = dl.attribute(input_, baseline, target=0, return_convergence_delta=True)
print('DeepLift Attributions:', attributions)
print('Convergence Delta:', delta)

In [ ]:
dl = DeepLiftShap(model)
attributions, delta = dl.attribute(input_.float(), baseline_dist, target=0, return_convergence_delta=True)
print('DeepLiftSHAP Attributions:', attributions)
print('Convergence Delta:', delta)
print('Average delta per example:', torch.mean(delta.reshape(input.shape[0], -1), dim=1))

In [ ]:
ig = IntegratedGradients(model)
nt = NoiseTunnel(ig)
attributions, delta = nt.attribute(input_, nt_type='smoothgrad', stdevs=0.02, nt_samples=4,
      baselines=baseline, target=0, return_convergence_delta=True)
print('IG + SmoothGrad Attributions:', attributions)
print('Convergence Delta:', delta)
print('Average delta per example', torch.mean(delta.reshape(input.shape[0], -1), dim=1))

In [ ]:
nc = NeuronConductance(model, model.backbone)
attributions = nc.attribute(input_, neuron_selector=1, target=0)
print('Neuron Attributions:', attributions)

In [ ]:
lc = LayerConductance(model, model.backbone)
attributions, delta = lc.attribute(input_, baselines=baseline, target=0, return_convergence_delta=True)
print('Layer Attributions:', attributions)
print('Convergence Delta:', delta)